## Load modules

In [2]:
import xarray as xr
import numpy as np
import cdo
import copy

## Initialisation cdo

In [3]:
cdo = cdo.Cdo()

Declare paths

In [4]:
inputpath_raw = '/data/cburgard/MELT_PARAM_NONRESOLVED/raw/'
inputpath_interim = '/data/cburgard/MELT_PARAM_NONRESOLVED/interim/'
inputpath_NEMO = '/data/cburgard/PREPARE_FORCING/PREPARE_CAVITY_MASKS/raw/eORCA1.4.3_OpenSeas_OpenAllCav_ModStraights/'
inputpath_NEMO_interim = '/data/cburgard/PREPARE_FORCING/PREPARE_CAVITY_MASKS/interim/'

## Define grids

In [5]:
## source grid
fBedMachine_grid=inputpath_interim + 'basins_for_param_v5_raster_extrapolate_with_zmin_zmax_isfconc_withgrid.nc'
BedMachine_grid=xr.open_dataset(fBedMachine_grid)

## target grid
fNEMO_grid = inputpath_NEMO_interim + 'NEMO_gridT_eORCA1_cdo.nc'
ds_nemo_grid=xr.open_dataset(fNEMO_grid)

## Inputs files

In [6]:
# BedMachine state data
ds_BedMachine=xr.open_dataset(inputpath_NEMO_interim + 'BedMachine_v3_aggregated2km_allvars.nc')

# NEMO bedrock data
ds_nemo=xr.open_dataset(inputpath_NEMO + 'eORCA1.4.3_OpenSeas_OpenAllCav_ModStraights_domain_cfg.nc')

# Manage basins
## Interpolation of basin variable from Elmer to NEMO grid
**Method:** cdo nearest neighbourg

In [7]:
## define the basin dataset to interpolate
ds_BedMachine_basins=BedMachine_grid.basin_extrap.to_dataset()
ds_BedMachine_basins['lat_bnds']=BedMachine_grid.lat_bnds
ds_BedMachine_basins['lon_bnds']=BedMachine_grid.lon_bnds

## CDO inperpolation
### cdo.remapnn(f'{target_grid}',input=input_xrdataset, returnArray=variable_name_to_process)
NEMO_basins=cdo.remapnn(f'{fNEMO_grid}',input=ds_BedMachine_basins, returnArray='basin_extrap').squeeze()

In [8]:
NEMO_basins.shape

(331, 360)

## Interpolation back to Elmer from NEMO grid
**Why:** We need to do this in case some small basins have been removed during the nn interpolation to NEMO. Without this step, there is possibility that the sum of the area distribution seen by NEMO do not match the total area to parametrised in ELMER.  

In [9]:
ds_nemo_basins=copy.deepcopy(ds_nemo_grid)#.rename_vars({'dummy': 'basin_extrap'})
ds_nemo_basins['basin_extrap']= xr.DataArray(data=NEMO_basins, dims=['y','x'])

## CDO inperpolation
### cdo.remapnn(f'{target_grid}',input=input_xrdataset, returnArray=variable_name_to_process)
BM_basins=cdo.remapnn(f'{fBedMachine_grid}',input=ds_nemo_basins, returnArray='basin_extrap')

In [10]:
BedMachine_grid

<xarray.Dataset>
Dimensions:       (y: 3041, x: 3041, nv4: 4, ID: 134)
Coordinates:
    lon           (y, x) float64 ...
    lat           (y, x) float64 ...
  * ID            (ID) float64 0.0 2.0 3.0 4.0 5.0 ... 517.0 518.0 540.0 541.0
  * x             (x) float64 -3.04e+06 -3.038e+06 ... 3.038e+06 3.04e+06
  * y             (y) float64 -3.04e+06 -3.038e+06 ... 3.038e+06 3.04e+06
Dimensions without coordinates: nv4
Data variables:
    lon_bnds      (y, x, nv4) float64 ...
    lat_bnds      (y, x, nv4) float64 ...
    basin_extrap  (y, x) float64 ...
    z_min         (ID) float64 ...
    z_max         (ID) float64 ...
    z_perc01      (ID) float64 ...
    z_perc99      (ID) float64 ...
    isf_conc      (y, x) float64 ...
Attributes:
    CDI:          Climate Data Interface version 2.4.1 (https://mpimet.mpg.de...
    Conventions:  CF-1.6
    Source:       Modified file based on basins_for_param_v5_raster_extrapola...
    Creator:      C. Burgard and P. Mathiot (May-19-2025)
    history:      Mon May 19 17:49:42 2025: cdo setgrid,ISMIP6_grid_AIS_2000m...
    CDO:          Climate Data Operators version 2.4.1 (https://mpimet.mpg.de...

# compute area not resolved by NEMO
**Method:**
- conservative interpolation of Elmer floating cell to NEMO
- set to 1 value between 0 and 1 (ie area to parametrised in NEMO)
- conservative interpolation back to ELMER to give the cell that need to be included in the param and the area scale factor

In [7]:
# create clean data_set array (cdo is very sensitive)
ds_BedMachine_sftflf = xr.Dataset(
    data_vars=dict(
        sftflf=(["time","ncells"], ds_elmer.sftflf.values),
        time_instant=(["time"],ds_elmer.time_instant.values),
        lon_bnds=(["ncells","vertices"], ds_elmer_grid.lon_bnds.values),
        lat_bnds=(["ncells","vertices"], ds_elmer_grid.lat_bnds.values),
        time_instant_bnds=(["time","axis_nbounds"],ds_elmer.time_instant_bounds.values),
    ),
    coords=dict(
        lon=(["ncells"],ds_elmer_grid.lon.values,ds_elmer_grid.lon.attrs),
        lat=(["ncells"],ds_elmer_grid.lat.values,ds_elmer_grid.lat.attrs),
    ),
)

In [8]:
## CDO inperpolation
### cdo.remapcon(f'{target_grid}',input=input_xrdataset, returnArray=variable_name_to_process)
NEMO_sftflf=cdo.remapcon(f'{fNEMO_grid}', input=ds_elmer_sftflf, returnArray='sftflf').squeeze()

In [9]:
# set to 1 cell to parametrised
NEMO_cell_to_param=np.zeros(shape=NEMO_sftflf.shape)
NEMO_cell_to_param=np.where((NEMO_sftflf > 0) & (NEMO_sftflf<1),1,0)

# Prepare NEMO fields for Domain cfg (draft and bathy)
## Interpolate ELMER isf draft to NEMO
**Method:** Conservative interpolation via cdo instead of basic linear interpolation using Elmer capability.

In [10]:
# create clean data_set array (cdo is very sensitive)
ds_elmer_base = xr.Dataset(
    data_vars=dict(
        base=(["time","ncells"], ds_elmer.base.values),
        time_instant=(["time"],ds_elmer.time_instant.values),
        lon_bnds=(["ncells","vertices"], ds_elmer_grid.lon_bnds.values),
        lat_bnds=(["ncells","vertices"], ds_elmer_grid.lat_bnds.values),
        time_instant_bnds=(["time","axis_nbounds"],ds_elmer.time_instant_bounds.values),
    ),
    coords=dict(
        lon=(["ncells"],ds_elmer_grid.lon.values,ds_elmer_grid.lon.attrs),
        lat=(["ncells"],ds_elmer_grid.lat.values,ds_elmer_grid.lat.attrs),
    ),
)

NEMO_base=cdo.remapcon(f'{fNEMO_grid}', input=ds_elmer_base, returnArray='base').squeeze()

# set nan to 0
NEMO_base=np.where(np.isnan(NEMO_base),0,NEMO_base)

## Read NEMO bathymetry

In [11]:
NEMO_bathy=ds_nemo.bathy_metry.squeeze()

## Mask domain cfg input files
**Method:** Use NEMO_sftflt variable

In [12]:
NEMO_mask = np.where(NEMO_sftflf<1,0,1)
NEMO_base = NEMO_base*NEMO_mask
NEMO_bathy= NEMO_bathy*NEMO_mask

## Write file to mask isf draft and bathymetry before computing domain_cfg


In [13]:
# create dataset
ds_out = xr.Dataset(
    data_vars=dict(
        isf_param_msk=(["x","y"], NEMO_cell_to_param ,dict(unit='[]',long_name='mask_unresolved_isf_cavities')),
        isf_draft=(["x","y"], NEMO_base  ,dict(unit='[]',long_name='mask isf draft from Elmer/Ice')),
        oce_bathy=(["x","y"], NEMO_bathy ,dict(unit='[]',long_name='mask ocean bathymetry')),
    ),
    coords=dict(
        lon=(["x","y"],ds_nemo_grid.lon.values.squeeze()),
        lat=(["x","y"],ds_nemo_grid.lat.values.squeeze()),
    ),
)

# save to netcdf
ds_out.to_netcdf('NEMO_geometry.nc')

In [14]:
# update area using domain_cfg_output / cut script in 2 here (prepare isf_draft and bathy and prepare param file)

# Interpolation back from NEMO to ELMER to retreive the area scale factor

In [15]:
# Read NEMO_geometry.nc
ds_nemo_geometry=xr.open_dataset('NEMO_geometry.nc')
NEMO_cell_to_param=ds_nemo_geometry.isf_param_msk.squeeze()

# Read domain_cfg.nc
ds_nemo_domain=xr.open_dataset('eORCA1.L121_modipsl_test_domain_cfg.nc')
NEMO_isfd=ds_nemo_geometry.isf_draft.squeeze()

# mask NEMO_geometry.nc 
# => set to 1 area that where floating that are now grounded using the base variable sent to domain cfg
NEMO_cell_to_param = np.where(NEMO_isfd == 0, 0, NEMO_cell_to_param)

# interpolation back to Elmer to retreive the area scale factor
ds_nemo_param=copy.deepcopy(ds_nemo_grid).rename_vars({'dummy': 'cell_to_param'})
ds_nemo_param['cell_to_param'].values=NEMO_cell_to_param.squeeze()

## CDO inperpolation
### cdo.remapcon(f'{target_grid}',input=input_xrdataset, returnArray=variable_name_to_process)
ELMER_cell_to_param_sf=cdo.remapcon(f'{fELMER_grid}', input=ds_nemo_param, returnArray='cell_to_param').squeeze()

## Compute depth distribution

In [16]:
# get elmer element area, ice shelf draft and floating cell mask
isfd=ds_elmer.base.values.squeeze()
mask_isf=ds_elmer.sftflf.values.squeeze()
cell_area=np.float64(ds_elmer.cell_area.values.squeeze())

# get NEMO depth range
e3t=ds_nemo_domain.e3t_1d.values.squeeze()

In [17]:
# compute weight for histogram
# As there is no melt in Elmer on the partially grounded element, these element should be excluded
ELMER_cell_to_param_sf[np.isnan(ELMER_cell_to_param_sf) | (mask_isf < 1)]=0.0
weight=np.float64(cell_area)*np.float64(ELMER_cell_to_param_sf)

# define histogram bin
binbnds=np.zeros(shape=(e3t.shape[0]+1,))
binbnds[1::]=e3t[:]

# retreive basin list id
basin_list=set(ELMER_basins.flatten())

# compute historgram for each basin
hist_isfd=np.zeros(shape=(len(basin_list),len(e3t)))
for ib, ibasin in enumerate(basin_list):
    hist_isfd[ib,:],_=np.histogram(-isfd[ELMER_basins==ibasin],binbnds,weights=weight[ELMER_basins==ibasin])

In [ ]:
# sum of (weights * isf_conc) should be total area of the ice shelf (dans chaque profondeur)

## Define the cell where to activate the param in NEMO

In [18]:
# Define NEMO coastal mask of the area to parametrised
def get_coastal_msk(mask_in,lewp):
    nj_out=mask_in.shape[0]
    ni_out=mask_in.shape[1]

    if lewp:
        mask=np.zeros(shape=(nj_out,ni_out+2))
        mask[:,1:-1]=mask_in
        mask[:, 0]=mask[:,-2]
        mask[:,-1]=mask[:, 1]
        xslc=slice(0,ni_out)
    else:
        mask=mask_in
        xslc=slice(1,-1)
        
    mask_coast=np.zeros(shape=(nj_out,ni_out))
    mask_coast[1:-1,xslc]= ( mask[1:-1,1:-1] + 
                             mask[0:-2,1:-1] + mask[2::,1:-1] + mask[1:-1,0:-2] + mask[1:-1,2::] +
                             mask[0:-2,0:-2] + mask[2::,2::]  + mask[2::,0:-2]  + mask[0:-2,2::] ) * mask[1:-1,1:-1]
    mask_coast[(mask_coast > 1) & (mask_coast < 9)] = 10
    mask_coast[mask_coast!=10]=np.nan
    mask_coast=mask_coast.astype(np.float32)
    mask_coast[mask_coast==10]=1
    
    return mask_coast

In [19]:
# get coastal cell along area to parametrized
# method:
# - reverse the cell_to_param mask
# - detect contour
# - pick from contour only the ocean points
NEMO_mask_param_basins=np.zeros(shape=NEMO_cell_to_param.shape)
NEMO_mask_param_basins=np.where(NEMO_cell_to_param>0,0,1)
NEMO_mask_param_basins=get_coastal_msk(NEMO_mask_param_basins,True)
NEMO_mask_param_basins[np.isnan(NEMO_mask_param_basins)]=0

# get NEMO mask
NEMO_mask=ds_nemo_domain.bottom_level.squeeze()
NEMO_mask=np.where(NEMO_mask>0,1,0)

# get basin number for each coastal cell
NEMO_mask_param_basins=np.where(NEMO_mask == 1, NEMO_mask_param_basins, 0)
NEMO_mask_param_basins=NEMO_mask_param_basins*NEMO_basins

# get scale factor for each coastal cell
NEMO_mask_param_sf=np.zeros(shape=NEMO_cell_to_param.shape)
for ib, ibasin in enumerate(basin_list):
    npts=len(np.where(NEMO_mask_param_basins==ibasin)[0])
    if npts > 0:
        NEMO_mask_param_sf=np.where(NEMO_mask_param_basins==ibasin,1.0/npts,NEMO_mask_param_sf)

## Define the output file

In [20]:
# create clean data_set array (cdo is very sensitive)
ds_out = xr.Dataset(
    data_vars=dict(
        isf_param_sf=(["x","y"], NEMO_mask_param_sf    ,dict(unit='[]',long_name='isf_param_area_scale_factor')),
        isf_param_id=(["x","y"], NEMO_mask_param_basins,dict(unit='[]',long_name='isf_param_basin_id')), # mettre le nouveau fichier de bassins
        isf_area_dist=(["id","z"], hist_isfd, dict(unit='m2',long_name='isf_draft_area_distribution_per_depth_bin')), 
    ),
    coords=dict(
        lon=(["x","y"],ds_nemo_domain.glamt.values.squeeze()),
        lat=(["x","y"],ds_nemo_domain.gphit.values.squeeze()),
        bin_width=(["z"],e3t),
        basins=(["id"],np.array(list(basin_list))),
    ),
)

# save to netcdf
ds_out.to_netcdf('toto.nc')